# IMPORTS

In [31]:
import math
import warnings
import numpy  as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from IPython.core.display import HTML
import re
import unicodedata

warnings.filterwarnings( 'ignore' )

##  Helper Functions

In [57]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [16, 8]
#    plt.rcParams['font.size'] = 24   
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.set_option( 'display.expand_frame_repr', False )
    pd.set_option('max_colwidth',200)
    pd.set_option('display.max_rows',2000)
    pd.set_option('display.max_columns',50)
    pd.set_option('display.float_format',lambda x: '{:,.2f}'.format(x) )   

    
def remove_special_characters(text):
    """
    This function remove special characters common to brazilian portuguese language such as '^~$ç and was based on
    http://stackoverflow.com/a/517974/3464573
    """
    # Unicode normalize transforms a character into its Latin's like.
    nfkd = unicodedata.normalize('NFKD', text)
    newtext = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    return re.sub('[^a-zA-Z0-9 \\\]', '', newtext)


def classe_imovel(x):
    #return print(type(title))
    if 'Casa' in x:
        return 'House'
    if 'Sobrado' in x:
        return 'House'
    if 'Lote' in x:
        return 'Allotment'
    if 'Apartamento' in x:
        return 'Apartment'
    if 'Chacara' in x:
        return 'Country house'
    if 'Sala' in x:
        return 'Comercial'    
    if 'Comercial' in x:
        return 'Commercial'    
    
    else:
        return 'Outro'
    
def info(x):

    data = pd.DataFrame({'Columns' : x.columns,
                         'Dtypes' :  x.dtypes,
                         '# observ.': x.shape[0],
                         "# unique": x.nunique(),
                         '# nulls' : x.isna().sum(),
                         '% nulls' : x.isna().sum()/x.shape[0]*100
                        }).reset_index()    
    return print(data)    

In [33]:
jupyter_settings();

Populating the interactive namespace from numpy and matplotlib


## Loading data

In [34]:
df_imoveis_raw = pd.read_csv( '../data/aldeia_realestate.csv')
df_condo_raw = pd.read_excel( '../data/condominium_x_streets.xlsx', sheet_name = 'condo')

In [35]:
df1=df_imoveis_raw.copy()

# Transformation

## Deduplicating, cleansing, categorizing and filtering

In [36]:
# delete duplicates url
df1.drop_duplicates(subset=['url'], inplace=True)

In [37]:
# Remove special characters
string_cols = df1.select_dtypes(include='object').columns
string_cols = string_cols[ (string_cols != 'address') & (string_cols != 'url')]
for col in string_cols:
    df1[col]=df1[col].apply([lambda x: remove_special_characters(x)]) 

In [38]:
# Classify real state per type
df1['type'] = df1['title'].apply(classe_imovel)
df1 = df1[df1['type'] != 'Outro']

In [39]:
# Clean string in numeric fields

df1['area_m2']=df1['area_m2'].str.replace('Area', '').str.strip()
df1['area_m2']=df1['area_m2'].str.replace('m2', '').str.strip()

df1['bedrooms']=df1['bedrooms'].str.replace('Quartos', '').str.strip()
df1['bedrooms']=df1['bedrooms'].str.replace('Quarto', '').str.strip()

df1['en_suites']=df1['en_suites'].str.replace('Suites', '').str.strip()
df1['en_suites']=df1['en_suites'].str.replace('Suite', '').str.strip()

df1['price']=df1['price'].str.replace('R', '').str.strip()
df1['price']=df1['price'].str.replace('Sob Consulta', '').str.strip()
df1['price']=df1['price'].str.replace('A partir de', '').str.strip()

df1['bathrooms']=df1['bathrooms'].str.replace('Banheiros', '').str.strip()
df1['bathrooms']=df1['bathrooms'].str.replace('Banheiro', '').str.strip()

df1['parking_spaces']=df1['parking_spaces'].str.replace('Vagas', '').str.strip()
df1['parking_spaces']=df1['parking_spaces'].str.replace('Vaga', '').str.strip()

In [40]:
# Keep only the three cities possible for Aldeia da Serra region
filter0 =  df1.address.str.contains('Barueri') | df1.address.str.contains('Santana') | \
           df1.address.str.contains('Itapevi')
df1 = df1[filter0]

## Splitting address information

### Cleaning up and prepare address to be splitted 

In [41]:
# street_name, street_number, condo_name and city_name
df1.address = df1.address.str.replace('\n', '')
df1['address_tmp'] = df1['address'].str.replace('\n','').str.strip()

# replace city separator from ',' to '@'
df1.address_tmp = df1.address_tmp.str.replace(', Barueri', '@ Barueri')
df1.address_tmp = df1.address_tmp.str.replace(', Santana', '@ Santana')
df1.address_tmp = df1.address_tmp.str.replace(', Itapevi', '@ Itapevi')

# exclude 'Aldeia da Serra'
df1.address_tmp = df1.address_tmp.str.replace('Aldeia da Serra', '')

df1.address_tmp = df1.address_tmp.str.replace('-', '')
df1.address_tmp = df1.address_tmp.str.replace('  ', ' ')

# exclude 'Residencial' to standardize condo name
df1.address_tmp = df1.address_tmp.str.replace('Residencial', '')

# insert condo name separator to '@'
df1.address_tmp = df1.address_tmp.str.replace('Morada', '@Morada')

### Identifying different  patterns on address field

In [42]:
#Create count_at(@) and count_comma to make splitting possible for every pattern
df1['count_at'] = df1.address_tmp.str.count('@')
df1['count_comma'] = df1.address_tmp.str.count(',')
df1.groupby(['count_at', 'count_comma'])['page_seq'].count()

count_at  count_comma
1         0              490
          1               63
2         0              734
          1              198
Name: page_seq, dtype: int64

In [43]:
# Demonstrating the four patterns for address
# street name, neighbourhood and city
sample_addr_t1 = df1[ (df1['count_at'] == 1) & (df1['count_comma'] == 0) ]['address'].sample(1)
# street name, street number, condo name and city
sample_addr_t2 = df1[ (df1['count_at'] == 1) & (df1['count_comma'] == 1) ]['address'].sample(1)
# street name, condo name and city
sample_addr_t3 = df1[ (df1['count_at'] == 2) & (df1['count_comma'] == 0) ]['address'].sample(1)
# street name, street number, condo name, neighbourhood and city
sample_addr_t4 = df1[ (df1['count_at'] == 2) & (df1['count_comma'] == 1) ]['address'].sample(1)                                                                          
#concat dfs
sample_addr_types = pd.concat([sample_addr_t1, sample_addr_t2, sample_addr_t3, sample_addr_t4])
print(sample_addr_types)

314                        Alameda das Begônias - Aldeia da Serra, Santana de Parnaíba - SP              
1545                    Estrada Doutor Yojiro Takaoka, 2165 - Aldeia da Serra, Barueri - SP              
1357                                                      Morada dos Pássaros, Barueri - SP              
1089                     Avenida Queimada, 369 - Residencial Morada dos Lagos, Barueri - SP              
Name: address, dtype: object


### Inputing '@' separator according to pattern

In [44]:
#desired format  : street_name '@' street_number '@' condo_name '@' city

#found format: street_name '@' city
#replace '@' by '@@@'
filter1 = (df1['count_at'] == 1) & (df1['count_comma'] == 0)
address_type1 = df1[filter1]['address_tmp'].str.replace('@','@ @ @')

#found format: street_name ',' '@' city
#replace '@' by '@@' and ',' by '@'
filter2 = (df1['count_at'] == 1) & (df1['count_comma'] == 1)
address_type2 = df1[filter2]['address_tmp'].str.replace('@',' @ @ ').str.replace(',',' @ ')

#found format: street_name '@' condo_name '@' city
#replace '@Morada' by '@ @Morada'
filter3 = (df1['count_at'] == 2) & (df1['count_comma'] == 0)
address_type3 = df1[filter3]['address_tmp'].str.replace('@Morada',' @ @Morada ')

#found format: street_name ',' street_number '@' condo_name '@' city
#replace ',' by '@'
filter4 = (df1['count_at'] == 2) & (df1['count_comma'] == 1)
address_type4 = df1[filter4]['address_tmp'].str.replace(',',' @ ')
new_address = pd.concat([address_type1,address_type2,address_type3,address_type4], axis=0).to_frame()

#new_address['street_name'], new_address['street_number'],new_address['condo'], new_address['city'] 
new_address = new_address['address_tmp'].str.split('@', n=3, expand=True)
new_address.columns=['street_name', 'street_number', 'condo_name', 'city']
cols = new_address.columns

In [45]:
#trim strings
for col in cols:
    new_address[col]=new_address[col].apply([lambda x: x.strip()]) 
    
#concatente features into the original dataset    
df1 = pd.concat([df1, new_address], axis=1)
df1.street_number = df1.street_number.str.replace('sn', '')

### Looking up condo_name base on street_name x condo_name dataset previously created

In [46]:
# merging dataset to get condo name through street_name
df1 = df1.merge(df_condo_raw, on='street_name', how='left')

In [47]:
#replacing lookup condo name for the original when the prior is blank
df1['condo_name_y'].fillna(df1['condo_name_x'], inplace=True)

In [48]:
df1.rename(columns={'condo_name_y': 'condo_name'}, inplace=True)

df1['condo_name'].replace('Morada  dos Lagos', 'Morada dos Lagos', inplace = True)
df1['condo_name'].replace('Morada  dos Pássaros', 'Morada dos Pássaros', inplace = True)
df1['condo_name'].replace('Morada  dos Pinheiros', 'Morada dos Pinheiros', inplace = True)

In [49]:
df1.drop(columns='condo_name_x', inplace = True)

### Converting numeric fields that are now cleaned

In [50]:
df1['area_m2'] = pd.to_numeric(df1['area_m2'])
df1['bedrooms'] = pd.to_numeric(df1['bedrooms'])
df1['bathrooms'] = pd.to_numeric(df1['bedrooms'])
df1['en_suites'] = pd.to_numeric(df1['en_suites'])
df1['parking_spaces'] = pd.to_numeric(df1['parking_spaces'])
df1['price'] = pd.to_numeric(df1['price'])
df1.street_number = df1.street_number.str.replace('483/503', '483')
df1['street_number'] = pd.to_numeric(df1['street_number'])

### Demonstrating before and after address splitting to each pattern

In [51]:
sample_addr_t1 = df1[ (df1['count_at'] == 1) & (df1['count_comma'] == 0) ][['address', 'address_tmp', \
                                            'street_name','street_number','condo_name', 'city']].head(1)

sample_addr_t2 = df1[ (df1['count_at'] == 1) & (df1['count_comma'] == 1) ][['address', 'address_tmp', \
                                            'street_name','street_number','condo_name', 'city']].head(1)

sample_addr_t3 = df1[ (df1['count_at'] == 2) & (df1['count_comma'] == 0) ][['address', 'address_tmp', \
                                            'street_name','street_number','condo_name', 'city']].head(1)

sample_addr_t4 = df1[ (df1['count_at'] == 2) & (df1['count_comma'] == 1) ][['address', 'address_tmp', \
                                            'street_name','street_number','condo_name', 'city']].head(1)
sample_addr_types = pd.concat([sample_addr_t1, sample_addr_t2, sample_addr_t3, sample_addr_t4])

sample_addr_types

,address,address_tmp,street_name,street_number,condo_name,city
0,"Alameda Colibri - Aldeia da Serra, Barueri - SP",Alameda Colibri @ Barueri SP,Alameda Colibri,nan,Morada dos Pássaros,Barueri SP
2,"Estrada Doutor Yojiro Takaoka, 2165 - Aldeia da Serra, Barueri - SP","Estrada Doutor Yojiro Takaoka, 2165 @ Barueri SP",Estrada Doutor Yojiro Takaoka,"2,165.00",Morada da Serra / Nuvens,Barueri SP
522,"Alameda das Dálias - Morada dos Pinheiros Aldeia da Serra , Santana de Parnaíba - SP",Alameda das Dálias @Morada dos Pinheiros @ Santana de Parnaíba SP,Alameda das Dálias,nan,Morada das Flores,Santana de Parnaíba SP
523,"Alameda das Margaridas, 230 - Morada dos Pinheiros Aldeia da Serra , Santana de Parnaíba - SP","Alameda das Margaridas, 230 @Morada dos Pinheiros @ Santana de Parnaíba SP",Alameda das Margaridas,230.00,Morada das Flores,Santana de Parnaíba SP


### Dropping auxiliary columns

In [52]:
df1.drop(columns=['count_at', 'count_comma', 'address_tmp'], inplace=True)

In [53]:
df1.sample(2)

,page_seq,house_seq,title,url,address,area_m2,bedrooms,en_suites,price,bathrooms,parking_spaces,type,street_name,street_number,city,condo_name
36,1,6,Casa de condominio com 3 Quartos a Venda 250m2,www.vivareal.com.br/imovel/casa-de-condominio-3-quartos-aldeia-da-serra-bairros-santana-de-parnaiba-com-garagem-250m2-venda-RS1060000-id-2482349928/,"Alameda dos Cravos - Aldeia da Serra, Santana de Parnaíba - SP",250.00,3.00,1.00,1060000,3.00,4.00,House,Alameda dos Cravos,nan,Santana de Parnaíba SP,Morada das Flores
532,0,12,Casa com 3 Quartos a Venda 468m2,www.vivareal.com.br/imovel/casa-3-quartos-morada-dos-pinheiros-aldeia-da-serra-bairros-santana-de-parnaiba-com-garagem-468m2-venda-RS1280000-id-2481315214/?__vt=plp:b,"Morada dos Pinheiros Aldeia da Serra , Santana de Parnaíba - SP",468.00,3.00,3.00,1280000,3.00,4.00,House,,nan,Santana de Parnaíba SP,Morada dos Pinheiros


In [58]:
info(df1)

             index         Columns   Dtypes  # observ.  # unique  # nulls  % nulls
0         page_seq        page_seq    int64       1485        15        0     0.00
1        house_seq       house_seq    int64       1485        36        0     0.00
2            title           title   object       1485       758        0     0.00
3              url             url   object       1485      1485        0     0.00
4          address         address   object       1485       401        0     0.00
5          area_m2         area_m2  float64       1485       394        2     0.13
6         bedrooms        bedrooms  float64       1485         9      253    17.04
7        en_suites       en_suites  float64       1485         8      274    18.45
8            price           price    int64       1485       357        0     0.00
9        bathrooms       bathrooms  float64       1485         9      253    17.04
10  parking_spaces  parking_spaces  float64       1485        17      279    18.79
11  

### Saving file

In [55]:
df1.to_csv('../data/cleaned_aldeia_realestate.csv', index=None)